# 🚀 AgentCore Valkey Checkpointer Usage Example

This notebook demonstrates how to use the **AgentCoreValkeySaver** for high-performance checkpoint storage that combines AgentCore session management concepts with Valkey's fast storage capabilities.

## 📋 Prerequisites

Before running this notebook, ensure you have:

1. **Valkey server** running on `localhost:6379`
2. **AWS credentials** configured for Bedrock access
3. **Required packages** installed:
   ```bash
   pip install valkey langchain-aws langchain langgraph
   ```

## 🎯 Key Features

- 🚀 **High-performance storage** with Valkey
- 🎯 **AgentCore-compatible** session management
- 🔄 **TTL support** for automatic cleanup
- 🏊 **Connection pooling** for scalability
- 🔍 **Metadata filtering** and querying
- 🧹 **Easy cleanup** and management

## 📦 Import Dependencies

First, let's import all the necessary libraries:

In [1]:
import time
from typing import Dict, Any

from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langgraph.prebuilt import create_react_agent

# Note: If you see a Pylance import error below, it's a false positive.
# The import works correctly at runtime - the package is properly installed.
from langgraph_checkpoint_aws import AgentCoreValkeySaver

print("✅ All dependencies imported successfully!")

✅ All dependencies imported successfully!


## 🛠️ Define Agent Tools

Let's create some simple mathematical tools for our agent to use:

In [2]:
@tool
def add(a: int, b: int) -> int:
    """Add two integers and return the result."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the result."""
    return a * b

# Create tools list
tools = [add, multiply]

print(f"✅ Created {len(tools)} tools for the agent:")
for tool in tools:
    print(f"   - {tool.name}: {tool.description}")

✅ Created 2 tools for the agent:
   - add: Add two integers and return the result.
   - multiply: Multiply two integers and return the result.


## ⚙️ Configuration

Set up the configuration for Valkey connection and the language model:

In [3]:
# Configuration
VALKEY_URL = "valkey://localhost:6379"
MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

print("📋 Configuration:")
print(f"   - Valkey URL: {VALKEY_URL}")
print(f"   - Model ID: {MODEL_ID}")

📋 Configuration:
   - Valkey URL: valkey://localhost:6379
   - Model ID: us.anthropic.claude-3-7-sonnet-20250219-v1:0


## 🤖 Initialize Language Model

Initialize the Bedrock language model:

In [4]:
# Initialize the language model
print("📝 Initializing language model...")

try:
    llm = init_chat_model(
        MODEL_ID,
        model_provider="bedrock",
        region_name="us-west-2"
    )
    print("✅ Language model initialized successfully!")
except Exception as e:
    print(f"❌ Failed to initialize model: {e}")
    print("Please check your AWS credentials and region configuration.")
    raise

📝 Initializing language model...
✅ Language model initialized successfully!


## 🔄 Create AgentCore Valkey Checkpointer

Create the checkpointer with TTL and connection pooling:

In [5]:
# Create AgentCore Valkey checkpointer with TTL
print("🔄 Connecting to Valkey and creating checkpointer...")

try:
    checkpointer = AgentCoreValkeySaver.from_conn_string(
        VALKEY_URL,
        ttl_seconds=3600,  # 1 hour TTL
        pool_size=10
    ).__enter__()  # Enter context manager
    
    print("✅ AgentCore Valkey checkpointer created successfully!")
    print(f"   - TTL: 3600 seconds (1 hour)")
    print(f"   - Pool size: 10 connections")
    
except Exception as e:
    print(f"❌ Failed to connect to Valkey: {e}")
    print("Please ensure Valkey server is running on localhost:6379")
    raise

🔄 Connecting to Valkey and creating checkpointer...
✅ AgentCore Valkey checkpointer created successfully!
   - TTL: 3600 seconds (1 hour)
   - Pool size: 10 connections


## 🤖 Create LangGraph Agent

Create the agent graph with our checkpointer:

In [6]:
# Create the agent graph with checkpointer
print("🤖 Creating LangGraph agent...")

graph = create_react_agent(
    model=llm,
    tools=tools,
    prompt="You are a helpful mathematical assistant.",
    checkpointer=checkpointer,
)

print("✅ LangGraph agent created successfully!")

🤖 Creating LangGraph agent...
✅ LangGraph agent created successfully!


/var/folders/z9/x36s18355_s5r7pqv88sc0680000gr/T/ipykernel_19452/2145319992.py:4: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = create_react_agent(


## 🎯 Configure Session

Set up the session configuration with both `thread_id` and `actor_id` (required for AgentCore checkpointer):

In [7]:
# Configuration for session management
config = {
    "configurable": {
        "thread_id": "demo-session-1",    # Session ID
        "actor_id": "user-123",           # Required for AgentCore checkpointer
    }
}

print("📋 Session configuration:")
for key, value in config["configurable"].items():
    print(f"   - {key}: {value}")

📋 Session configuration:
   - thread_id: demo-session-1
   - actor_id: user-123


## 💬 First Conversation

Let's start a conversation with our agent:

In [8]:
print("💬 Starting conversation...")
print("\n👤 User: What is 15 multiplied by 23, then add 100?")

inputs = {
    "messages": [{
        "role": "user",
        "content": "What is 15 multiplied by 23, then add 100?"
    }]
}

print("🔄 Processing...")
response = graph.invoke(inputs, config)

assistant_message = response["messages"][-1].content
print(f"🤖 Assistant: {assistant_message}")

💬 Starting conversation...

👤 User: What is 15 multiplied by 23, then add 100?
🔄 Processing...
🤖 Assistant: The result of 15 multiplied by 23, then adding 100, is 445.


## 📋 Checkpoint State Inspection

Let's examine the current checkpoint state:

In [9]:
# Demonstrate checkpoint retrieval
print("📋 Current checkpoint state:")
state = graph.get_state(config)

checkpoint_id = state.config['configurable']['checkpoint_id']
message_count = len(state.values.get('messages', []))

print(f"   - Checkpoint ID: {checkpoint_id}")
print(f"   - Number of messages: {message_count}")
print(f"   - State keys: {list(state.values.keys())}")

📋 Current checkpoint state:
   - Checkpoint ID: 1f0b5047-5f6e-6156-8005-c4f7a8d26453
   - Number of messages: 6
   - State keys: ['messages']


## 🔄 Continue Conversation (Memory Test)

Let's continue the conversation to test memory persistence:

In [10]:
# Continue conversation (demonstrates memory persistence)
print("👤 User: Now divide that result by 5")

inputs2 = {
    "messages": [{
        "role": "user",
        "content": "Now divide that result by 5"
    }]
}

print("🔄 Processing...")
response2 = graph.invoke(inputs2, config)

assistant_message2 = response2["messages"][-1].content
print(f"🤖 Assistant: {assistant_message2}")

👤 User: Now divide that result by 5
🔄 Processing...
🤖 Assistant: I need to divide the previous result (445) by 5. However, I don't have a built-in division function available among the tools. I can do this calculation for you manually.

The result of dividing 445 by 5 is:
445 ÷ 5 = 89

So, after dividing 445 by 5, the answer is 89.


## 📚 Checkpoint History

View the checkpoint history for this session:

In [11]:
# Show checkpoint history
print("📚 Checkpoint history:")
history = list(graph.get_state_history(config, limit=3))

for i, checkpoint in enumerate(history):
    msg_count = len(checkpoint.values.get("messages", []))
    checkpoint_id = checkpoint.config["configurable"]["checkpoint_id"]
    print(f"   {i+1}. Checkpoint {checkpoint_id[:8]}... ({msg_count} messages)")

print(f"\n   Total checkpoints in history: {len(history)}")

📚 Checkpoint history:
   1. Checkpoint 1f0b5047... (8 messages)
   2. Checkpoint 1f0b5047... (7 messages)
   3. Checkpoint 1f0b5047... (6 messages)

   Total checkpoints in history: 3


## 🆕 New Session Test

Create a new session to demonstrate session isolation:

In [12]:
# Demonstrate new session
new_session_config = {
    "configurable": {
        "thread_id": "demo-session-2",    # New session
        "actor_id": "user-456",           # Required for AgentCore checkpointer
    }
}

print("🆕 Starting new session (no memory of previous conversation)...")
print("👤 User: What was my previous calculation?")

inputs3 = {
    "messages": [{
        "role": "user",
        "content": "What was my previous calculation?"
    }]
}

response3 = graph.invoke(inputs3, new_session_config)
assistant_message3 = response3["messages"][-1].content
print(f"🤖 Assistant: {assistant_message3}")

🆕 Starting new session (no memory of previous conversation)...
👤 User: What was my previous calculation?
🤖 Assistant: I don't have any record of your previous calculations. This appears to be our first interaction in this session. 

To help you with calculations, I can:
- Add two integers using the `add` function
- Multiply two integers using the `multiply` function

Would you like to perform one of these calculations? If so, please provide the numbers you'd like to calculate with.


## 📊 Storage Statistics

Let's examine what's stored in Valkey:

In [13]:
# Show storage statistics
print("📊 Storage Information:")

# Count keys in Valkey - using general patterns since we don't know the exact namespace
all_keys = checkpointer.client.keys("*")
session_keys = [k for k in all_keys if b"session" in k]
checkpoint_keys = [k for k in all_keys if b"checkpoint" in k]
writes_keys = [k for k in all_keys if b"writes" in k]

print(f"   - Total keys: {len(all_keys)}")
print(f"   - Session-related keys: {len(session_keys)}")
print(f"   - Checkpoint-related keys: {len(checkpoint_keys)}")
print(f"   - Writes-related keys: {len(writes_keys)}")

# Show some example keys
if all_keys:
    print(f"\n   Example keys:")
    for i, key in enumerate(all_keys[:3]):
        print(f"     {i+1}. {key.decode()}")

📊 Storage Information:
   - Total keys: 47
   - Session-related keys: 47
   - Checkpoint-related keys: 15
   - Writes-related keys: 10

   Example keys:
     1. agentcore:channel:demo-session-1:user-123::branch:to:agent:1f0b5047-1ca4-6258-8002-313023f32ae0
     2. agentcore:checkpoint:demo-session-1:user-123::1f0b5046-c952-679e-bfff-945450391b51
     3. agentcore:checkpoint:demo-session-1:user-123::1f0b5047-b7a3-6ef2-8006-a6027f7fdc14


## 🧹 Cleanup Demonstration

Demonstrate cleanup functionality:

In [14]:
# Demonstrate cleanup - AgentCore checkpointer requires both thread_id and actor_id
print("🧹 Cleaning up sessions...")

# For AgentCore checkpointer, we need to provide both thread_id and actor_id
try:
    # Clean up session 2 - need both thread_id and actor_id
    checkpointer.delete_thread("demo-session-2", "user-456")
    print("   ✅ Session 2 cleaned up successfully!")
except Exception as e:
    print(f"   ⚠️  Error cleaning session 2: {e}")

# Verify cleanup by checking remaining keys
remaining_keys = checkpointer.client.keys("*demo-session-2*")
print(f"   - Remaining keys for session 2: {len(remaining_keys)}")

🧹 Cleaning up sessions...
   ✅ Session 2 cleaned up successfully!
   - Remaining keys for session 2: 0


## 🔬 Advanced Features Demo

Let's demonstrate some advanced features like connection pooling:

In [15]:
print("🔬 Advanced Features Demo")
print("=" * 30)

try:
    # Using connection pool for better performance
    from valkey.connection import ConnectionPool

    print("🏊 Creating connection pool...")
    pool = ConnectionPool.from_url(
        "valkey://localhost:6379",
        max_connections=20,
        retry_on_timeout=True
    )

    # Create checkpointer with pool
    pool_checkpointer = AgentCoreValkeySaver.from_pool(
        pool,
        ttl_seconds=1800  # 30 minutes
    ).__enter__()
    
    print("✅ Connection pool checkpointer created!")
    print(f"   - Max connections: 20")
    print(f"   - TTL: 1800 seconds (30 minutes)")
    
except Exception as e:
    print(f"❌ Advanced features error: {e}")
    pool_checkpointer = checkpointer  # Fallback to basic checkpointer

🔬 Advanced Features Demo
🏊 Creating connection pool...
✅ Connection pool checkpointer created!
   - Max connections: 20
   - TTL: 1800 seconds (30 minutes)


## 🤖 Multi-Session Scenario

Demonstrate multiple sessions working with the same checkpointer:

In [16]:
# Multiple sessions with different thread IDs and actor IDs
session_configs = [
    {
        "configurable": {
            "thread_id": f"multi-session-{i}",
            "actor_id": f"user-{i+100}",  # Required for AgentCore checkpointer
        }
    }
    for i in range(3)
]

print(f"🤖 Creating {len(session_configs)} sessions...")

# Test each session with a simple interaction
for i, config in enumerate(session_configs):
    inputs = {
        "messages": [{
            "role": "user",
            "content": f"Hello, I'm user {i}. What is {i+1} times 10?"
        }]
    }
    
    try:
        response = graph.invoke(inputs, config)
        print(f"   ✅ Session {i} created and tested")
    except Exception as e:
        print(f"   ❌ Session {i} failed: {e}")

print("✅ All sessions created")

🤖 Creating 3 sessions...
   ✅ Session 0 created and tested
   ✅ Session 1 created and tested
   ✅ Session 2 created and tested
✅ All sessions created


## 🔍 Session Inspection

Demonstrate querying capabilities:

In [17]:
# Demonstrate session inspection
print("🔍 Inspecting sessions...")

for i, config in enumerate(session_configs):
    try:
        state = graph.get_state(config)
        message_count = len(state.values.get("messages", []))
        thread_id = config["configurable"]["thread_id"]
        print(f"   Session {thread_id}: {message_count} messages")
    except Exception as e:
        print(f"   ❌ Error inspecting session {i}: {e}")

🔍 Inspecting sessions...
   Session multi-session-0: 4 messages
   Session multi-session-1: 4 messages
   Session multi-session-2: 4 messages


## 🧹 Final Cleanup

Clean up all the demo data:

In [18]:
# Cleanup all demo sessions - AgentCore requires both thread_id and actor_id
print("🧹 Cleaning up all demo sessions...")

# Clean up multi-sessions - need both thread_id and actor_id
for i in range(3):
    try:
        checkpointer.delete_thread(f"multi-session-{i}", f"user-{i+100}")
        print(f"   ✅ Cleaned up multi-session-{i}")
    except Exception as e:
        print(f"   ⚠️  Error cleaning multi-session-{i}: {e}")

# Clean up original demo session - need both thread_id and actor_id
try:
    checkpointer.delete_thread("demo-session-1", "user-123")
    print("   ✅ Cleaned up demo-session-1")
except Exception as e:
    print(f"   ⚠️  Error cleaning demo-session-1: {e}")

print("✅ All cleanup complete")

🧹 Cleaning up all demo sessions...
   ✅ Cleaned up multi-session-0
   ✅ Cleaned up multi-session-1
   ✅ Cleaned up multi-session-2
   ✅ Cleaned up demo-session-1
✅ All cleanup complete


## 🔒 Close Connections

Properly close all connections:

In [19]:
# Close connections properly
try:
    checkpointer.__exit__(None, None, None)
    print("✅ Basic checkpointer connection closed")
except:
    pass

try:
    if 'pool_checkpointer' in locals() and pool_checkpointer != checkpointer:
        pool_checkpointer.__exit__(None, None, None)
        print("✅ Pool checkpointer connection closed")
except:
    pass

print("✅ All connections closed successfully")

✅ All connections closed successfully


## 🎉 Summary

Congratulations! You've successfully demonstrated the **AgentCore Valkey Checkpointer**.

### Key Benefits Demonstrated:

- 🚀 **High-performance storage** with Valkey
- 🎯 **Session management** with thread IDs and actor IDs
- 🔄 **TTL support** for automatic cleanup
- 🏊 **Connection pooling** for scalability
- 🔍 **State inspection** and querying
- 🧹 **Easy cleanup** and management
- 📝 **JSON serialization** with base64 encoding
- 🔒 **Thread-safe** operations

### What We Covered:

1. **Basic Setup**: Created checkpointer with TTL and connection pooling
2. **Agent Integration**: Used with LangGraph agents
3. **Session Management**: AgentCore-compatible thread_id and actor_id patterns
4. **Memory Persistence**: Demonstrated conversation continuity
5. **Session Isolation**: Showed separate sessions don't share memory
6. **Storage Inspection**: Examined Valkey key structure
7. **Advanced Features**: Connection pooling and multi-session scenarios
8. **State Inspection**: Queried session states
9. **Cleanup**: Proper resource management and cleanup

### 🆚 AgentCore vs Standard LangGraph Differences:

| Operation | Standard LangGraph | AgentCore |
|-----------|-------------------|----------|
| **Session Config** | `{"thread_id": "session-1"}` | `{"thread_id": "session-1", "actor_id": "user-123"}` |
| **Cleanup** | `delete_thread("session-1")` | `delete_thread("session-1", "user-123")` |

The **AgentCoreValkeySaver** provides a powerful, scalable solution for checkpoint storage that works seamlessly with AgentCore session management patterns! 🚀